# 하나씩 차근차근 해보아요

# Algorithm
---

In [11]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
import os
# %matplotlib ipympl

In [12]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [13]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
'''
bond_energy = 2.07 
temperature = 700
e0 = 0.1

# site energy, e_(bond number)
e_site = np.zeros(4)
for i in range(4):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   


In [14]:
# value check
print(e_site)
print('-----------------')
print(e_a)
print('-----------------')
print(diffusion_rate)


[ 0.    -1.035 -2.07  -3.105]
-----------------
[[ 1.0000e-01 -3.5000e-03 -1.0700e-01 -2.1050e-01]
 [ 1.2385e+00  1.0000e-01 -3.5000e-03 -1.0700e-01]
 [ 2.3770e+00  1.2385e+00  1.0000e-01 -3.5000e-03]
 [ 3.5155e+00  2.3770e+00  1.2385e+00  1.0000e-01]]
-----------------
[[1.90559975e+12 1.05973894e+13 5.89340238e+13 3.27742904e+14]
 [1.21113472e+04 1.90559975e+12 1.05973894e+13 5.89340238e+13]
 [7.69756242e-05 1.21113472e+04 1.90559975e+12 1.05973894e+13]
 [4.89231019e-13 7.69756242e-05 1.21113472e+04 1.90559975e+12]]


In [15]:
# function to initialize the lattice
'''
Initialize the lattice
1 : atom
0 : vacancy or vacumm
2 more layer to make vacuum
'''
def init_lattice(width, height):
    lattice = np.ones((height+2, width), dtype=int)
    lattice[0, :] = 0
    lattice[-1, :] = 0

    return lattice

In [16]:
# Function to plot the lattice of circles
def plot_lattice(ax, width, height, atom_radius, lattice):
    for x in range(width):
        for y in range(height+2):
            if lattice[y, x] == 1:
                circle = plt.Circle((x + 0.5, y + 0.5), atom_radius, color='blue')
                ax.add_artist(circle)
    ax.set_xlim(0, width)
    ax.set_ylim(0, height+2)
    ax.set_aspect('equal')
    ax.axis('off')
    plt.tight_layout(pad=0.5)

In [17]:
# Function to draw lattice on prompt
def draw_lattice(lattice):
    height, width = lattice.shape
    for y in range(height):
        for x in range(width):
            if lattice[y, x] == 1:
                print('●', end = '')
            else:
                print(' ', end='')
        print('\n')

In [18]:
# Function to save lattice
def save_lattice(lattice, output_filename):
    # Check if the file already exists, and if so, remove it
    try:
        with open(output_filename, 'x'):
            pass
    except FileExistsError:
        pass

    # Perform iterative saving
   
    with open(output_filename, 'ab') as file:
        np.savetxt(file, lattice, fmt='%d', delimiter=' ')
        file.write(b'\n')

def save_matrix_as_xyz(matrix, output_filename):
    with open(output_filename, 'a') as output_file:
        num_atoms = int(np.sum(matrix))  # Calculate the number of Cu atoms (1s)

        # Write the number of atoms as the first line in the XYZ file
        output_file.write(f"{num_atoms}\n\n")

        # Iterate through the matrix to write the atom positions
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if matrix[i, j] == 1:  # Cu atom
                    output_file.write(f"Cu {i} {j}\n")

In [19]:
'''
function that finds every possible way
1. atom jumped from surface
2. return to previous position(jumped from surface)
3. other diffusion...
'''
def get_atoms_around_site(lattice, x, y, total=False):
    height, width = lattice.shape
    # calculate bond number
    if y == height-1:
        down = 0
    else:
        down = lattice[y+1, x]
    if y == 0:
        up = 0
    else:
        up = lattice[y-1, x]
    # PBC at left and right
    right = lattice[y, (x+1)%width]
    left = lattice[y, (x-1)%width]
    
    if total:
        return left+up+right+down
    else:
        return left, up, right, down

def find_candidate(lattice):
    global e_a
    global diffusion_rate
    height, width = lattice.shape
    candidate_table = []
    diffusion_table = []
    motion_table = []
    for x in range(width):
        for y in range(height):
            # find vacancy
            if lattice[y, x] == 0:
                left, up , right, down = get_atoms_around_site(lattice, x, y)
                neighbor = left+up+right+down
                if neighbor:
                    if left:
                        neighbor_of_left = get_atoms_around_site(lattice, (x-1)%width, y, True)
                        candidate_table.append((y, (x-1)%width))
                        motion_table.append(3)
                        diffusion_table.append(diffusion_rate[neighbor_of_left, neighbor-1])
                    if right:
                        neighbor_of_right = get_atoms_around_site(lattice, (x+1)%width, y, True)
                        candidate_table.append((y, (x+1)%width))
                        motion_table.append(1)
                        diffusion_table.append(diffusion_rate[neighbor_of_right, neighbor-1])
                    if up:
                        neighbor_of_up = get_atoms_around_site(lattice, x, y-1, True)
                        candidate_table.append((y-1, x))
                        motion_table.append(4)
                        diffusion_table.append(diffusion_rate[neighbor_of_up, neighbor-1])
                    if down:
                        neighbor_of_down = get_atoms_around_site(lattice, x, y+1, True)
                        candidate_table.append((y+1, x))
                        motion_table.append(2)
                        diffusion_table.append(diffusion_rate[neighbor_of_down, neighbor-1]) 
    
    return candidate_table, diffusion_table, motion_table

In [47]:
# KMC function
def diffuse_one_step(lattice, print_out=False):
    global time_elapsed
    width = lattice.shape[1]

    cand, dif, motion = find_candidate(lattice)
    dif = np.array(dif)

    total_dif = np.sum(dif)
    
    # pick 1
    u = np.random.uniform(low=1e-6, high=1)
    u_time = np.random.uniform(low=1e-6, high=1)
    cum_dif = np.cumsum(dif)

    chosen_idx = np.argwhere(u*total_dif < cum_dif)[0][0]

    # print information
    if print_out:
        print(f'total_diff : {total_dif}')
        print(f'chosen : {cand[chosen_idx]} atom')   
    # print(f'motion : {motion[chosen_idx]}')
    # print(cand[chosen_idx])
    # print(motion[chosen_idx])

    # change the lattice
    x, y = cand[chosen_idx][0], cand[chosen_idx][1]

    # get motion
    if motion[chosen_idx] == 1:
        lattice[x, y] = 0
        lattice[x, (y-1+width)%width] = 1
        if print_out:
            print(f'go left')   
    if motion[chosen_idx] == 2:
        lattice[x, y] = 0
        lattice[x-1 , y] = 1
        if print_out:
            print(f'go up')   
    if motion[chosen_idx] == 3:
        lattice[x, y] = 0
        lattice[x, (y+1)%width] = 1
        if print_out:
            print(f'go right')   
    if motion[chosen_idx] == 4:
        lattice[x, y] = 0
        lattice[x+1 , y] = 1
        if print_out:
            print(f'go down')   
    
    # time update
    delta_t = -np.log(u_time)/total_dif
    # print(f'delta_t : {delta_t}')
    time_elapsed += delta_t

## Animation using text(fast)
---

In [67]:
# Parameters
# Size of the lattice (height x width)
width = 100  
height = 10

# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
'''
bond_energy = 2.07 
temperature = 1300
e0 = 0.1

# site energy, e_(bond number)
e_site = np.zeros(4)
for i in range(4):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
  
steps = 100
time_elapsed = 0

In [83]:
num_closest_neighbors = 4
for e0 in [0, 0.1, 0.2, 0.3]:
    for temperature in [300, 500, 700, 1000, 1300]:
        # activation energy, e_a_(start to end)
        e_a = np.zeros((num_closest_neighbors, num_closest_neighbors))
        for i in range(num_closest_neighbors):
            for j in range(num_closest_neighbors):
                e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

        # diffusion rate, rate_(start to end)
        diffusion_rate = np.zeros((num_closest_neighbors, num_closest_neighbors))
        for i in range(num_closest_neighbors):
            for j in range(num_closest_neighbors):
                diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
        np.savetxt(f'2d_rate_const/rate_const_{temperature}K_{e0}.txt', diffusion_rate, fmt='%.2e')

In [69]:
# '''
# text-based simulation
# '''
# lattice = init_lattice(width, height)
# save_lattice(lattice, 'test100.txt')

# for i in range(1, steps+1):
#     print(f'---------------- step {i} ---------------------')
#     diffuse_one_step(lattice, True)
#     draw_lattice(lattice)
#     save_lattice(lattice, 'test100.txt')
#     print(f'Time elapsed : {time_elapsed} s')
#     time.sleep(0.4)
    
#     os.system('clear')
#     clear_output(wait=True)
    # fig, ax = plt.subplots(figsize=(10, 2))
    # plot_lattice(ax, width, height, atom_radius, lattice)


In [70]:
'''
simulation without visualization
Real simulation!
'''
# parameter tuning
steps = 1
width = 100
height = 10
save_file_name = '7by5_10000steps.xyz'
time_elapsed = 0

lattice = init_lattice(width, height)
# save_matrix_as_xyz(lattice, save_file_name)
atom_num = lattice.sum()
for i in range(1, steps+1):
    diffuse_one_step(lattice)
    # save_lattice(lattice, save_file_name)
    save_matrix_as_xyz(lattice, save_file_name)
    if i % 1000 == 0:
        print(f'---------------- step {i} ---------------------')
        print(f'Time elapsed : {time_elapsed} s')
        if lattice.sum() != atom_num:
            print('error!')
            break

# Analysis
---

In [77]:
'''
get average time_elapsed of surface jump
'''
steps = 100
width = 100
height = 10

for width in [10000, 100000]:
    for temp in [300, 500, 700, 1000, 1300]:
        diffusion_rate = np.zeros((4, 4))
        for i in range(4):
            for j in range(4):
                diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temp)
        time_elapsed = 0
        for i in range(1, steps+1):
            lattice = init_lattice(width, height)
            diffuse_one_step(lattice)
        print(f'Width:{width}|temp:{temp}| avg_time:{time_elapsed/steps} s')


Width:10000|temp:300| avg_time:4.782176916048693e+41 s
Width:10000|temp:500| avg_time:1.4273432204931246e+18 s
Width:10000|temp:700| avg_time:96701407.15836778 s
Width:10000|temp:1000| avg_time:2.2299932710245267 s
Width:10000|temp:1300| avg_time:0.0002320665867113364 s
Width:100000|temp:300| avg_time:6.047615960111374e+40 s
Width:100000|temp:500| avg_time:1.2715310268114811e+17 s
Width:100000|temp:700| avg_time:8161187.165094125 s
Width:100000|temp:1000| avg_time:0.298462384018557 s
Width:100000|temp:1300| avg_time:2.280001985847593e-05 s


In [82]:
'''
get average time_elapsed of surface jump, e0 change
'''
steps = 10000
width = 100
height = 10

for temp in [1300]:
    for e0 in [0, 0.1, 0.2, 0.3]:
        e_a = np.zeros((4, 4))
        for i in range(4):
            for j in range(4):
                e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)
        diffusion_rate = np.zeros((4, 4))
        for i in range(4):
            for j in range(4):
                diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temp)
        time_elapsed = 0
        for i in range(1, steps+1):
            lattice = init_lattice(width, height)
            diffuse_one_step(lattice)
        print(f'e0:{e0}|temp:{temp}| avg_time:{time_elapsed/steps} s')

e0:0|temp:1300| avg_time:0.008857778315523715 s
e0:0.1|temp:1300| avg_time:0.021428674912446256 s
e0:0.2|temp:1300| avg_time:0.05173701291724655 s
e0:0.3|temp:1300| avg_time:0.12576186522144053 s


# Start with vacancy
---

In [21]:
'''
start simulation with one vacancy
'''
# parameter tuning
steps = 100
width = 100
height = 10
time_elapsed = 0
'-----------------------------------------------------------------'


# initialize
lattice = init_lattice(width, height)

# make vacancy
lattice[5, 50] = 0
for i in range(1, steps+1):
    print(f'---------------- step {i} ---------------------')
    diffuse_one_step(lattice, True)
    draw_lattice(lattice)
    print(f'Time elapsed : {time_elapsed} s')
    time.sleep(0.4)
    
    os.system('clear')
    clear_output(wait=True)

---------------- step 100 ---------------------
total_diff : 417930372361.77185
chosen : (10, 67) atom
go left
                                                                                                    

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

In [47]:
# vacancy + no visualize
# parameter tuning
steps = 10000
width = 300
height = 20
time_elapsed = 0
'-----------------------------------------------------------------'

lattice = init_lattice(width, height)

# make vacancy
lattice[10, 150] = 0

atom_num = lattice.sum()
for i in range(1, steps+1):
    diffuse_one_step(lattice)
    if i % 1000 == 0:
        print(f'---------------- step {i} ---------------------')
        print(f'Time elapsed : {time_elapsed} s')
        if lattice.sum() != atom_num:
            print('error!')
            break

---------------- step 1000 ---------------------
Time elapsed : 2.5593605100587533e-10 s
---------------- step 2000 ---------------------
Time elapsed : 5.29326820110763e-10 s
---------------- step 3000 ---------------------
Time elapsed : 7.922695829227711e-10 s
---------------- step 4000 ---------------------
Time elapsed : 1.0562340177351715e-09 s
---------------- step 5000 ---------------------
Time elapsed : 1.3173568931482398e-09 s
---------------- step 6000 ---------------------
Time elapsed : 1.5835682176808373e-09 s
---------------- step 7000 ---------------------
Time elapsed : 1.8564033993476096e-09 s
---------------- step 8000 ---------------------
Time elapsed : 2.1282875956763153e-09 s
---------------- step 9000 ---------------------
Time elapsed : 2.3835161816815982e-09 s
---------------- step 10000 ---------------------
Time elapsed : 2.6584171177304157e-09 s


# step by step check

In [64]:
# init
width = 300
height = 20
lattice = init_lattice(width, height)

In [30]:
# step1
diffuse_one_step(lattice, True)

total_diff : 2.935386113111737e-10
chosen : (20, 151) atom
go down


In [31]:
draw_lattice(lattice)

                                                                                                                                                                                                                                                                                                            

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

In [45]:
# step2
diffuse_one_step(lattice, True)

total_diff : 7622399035571.027
chosen : (20, 152) atom


In [46]:
draw_lattice(lattice)

                                                                                                                                                                                                                                                                                                            

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

In [65]:
cand, dif, motion = find_candidate(lattice)

In [66]:
dif

[4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.892310188519563e-13,
 4.8923101885195

In [67]:
dif = np.array(dif)
total_dif = np.sum(dif)
u_time = np.random.uniform(low=1e-6, high=1)
delta_t = -np.log(u_time)/total_dif
print(delta_t)

3150303124.857064


In [68]:
-np.log(u_time)

0.9247356044797936

In [69]:
total_dif

2.935386113111737e-10

In [70]:
len(cand)

600

In [81]:
lattice = init_lattice(width, height)

In [86]:
save_lattice(lattice, 'test.txt')